# Rock, Paper, Scissor Hand Detection using CNN

**This project is a work in progress as a requirement for Dicoding Machine Learning Path**


- Name : Andi Dwi Prastyo
- College : state polytechnic of malang
- Major : Information Technology
- Study Program : D4 - Informatics Engineering

### Import Libraries

In [1]:
import os
import zipfile
import splitfolders

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-11-01 18:45:01.346448: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 18:45:01.594194: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 18:45:01.596055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 18:45:02.845759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Obtain Dataset

In [5]:
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

--2023-11-01 18:46:58--  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... failed: No route to host.
